In [1]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules, fpgrowth
from mlxtend.preprocessing import TransactionEncoder
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
try:
    df = pd.read_excel("OnlineRetail.xlsx")
    print("Dataset cargado exitosamente.")
    print("Dimensiones iniciales:", df.shape)
except FileNotFoundError:
    print("Error: Asegúrate de que el archivo 'OnlineRetail.xlsx' esté en el mismo directorio que tu notebook.")


print("Primeras filas del dataset:")
print(df.head())

Dataset cargado exitosamente.
Dimensiones iniciales: (541909, 8)
Primeras filas del dataset:
  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

          InvoiceDate  UnitPrice  CustomerID         Country  
0 2010-12-01 08:26:00       2.55     17850.0  United Kingdom  
1 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
2 2010-12-01 08:26:00       2.75     17850.0  United Kingdom  
3 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
4 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [8]:
df.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

# Limpieza del dataset

In [ ]:
# Eliminamos filas con nulos en 'CustomerID' y 'Description'
df.dropna(axis= 0, subset=['CustomerID', 'Description'], inplace=True)
print(f"Dimensiones tras eliminar nulos en CustomerID y Description: {df.shape}")

Dimensiones tras eliminar nulos en CustomerID y Description: (406829, 8)


In [12]:
# Convertimos 'CustomerID' a entero
df['CustomerID'] = df['CustomerID'].astype(int)

In [13]:
# Eliminamos transacciones con cantidad negativa o cero
df = df[df['Quantity'] > 0]
print(f"Dimensiones finales del dataset limpio: {df.shape}")

Dimensiones finales del dataset limpio: (397924, 8)


# Descripcion del dataset

In [15]:
# Número de transacciones (facturas únicas)
num_transacciones = df['InvoiceNo'].nunique()
print(f"Número de transacciones únicas: {num_transacciones}")

Número de transacciones únicas: 18536


In [16]:
# Número de ítems únicos
num_items = df['Description'].nunique()
print(f"Número de ítems únicos: {num_items}")

Número de ítems únicos: 3877


In [19]:
# Número de clientes únicos
num_clientes = df['CustomerID'].nunique()
print(f"Número de clientes únicos: {num_clientes}")

Número de clientes únicos: 4339


#División del Dataset

In [ ]:
# ============================================
# PREPARACIÓN DE DATOS
# ============================================

df['TotalPrice'] = df['Quantity'] * df['UnitPrice']

print("\n" + "="*70)
print("DIVISIÓN DEL DATASET EN PARTICIONES")
print("="*70)

# ============================================
# PARTICIÓN 1: TEMPORAL (por semestre)
# ============================================

print("\n" + "="*70)
print("PARTICIÓN 1: DIVISIÓN TEMPORAL (SEMESTRES)")
print("="*70)

# Crear columnas temporales
df['Year'] = df['InvoiceDate'].dt.year
df['Month'] = df['InvoiceDate'].dt.month
df['Semester'] = df['InvoiceDate'].dt.to_period('Q').apply(lambda x: 1 if x.quarter <= 2 else 2)
df['Year_Semester'] = df['Year'].astype(str) + '-S' + df['Semester'].astype(str)

# Dividir en semestres
df_semester_1 = df[df['Semester'] == 1].copy()
df_semester_2 = df[df['Semester'] == 2].copy()

print("\nJUSTIFICACIÓN:")
print("La división por semestres permite identificar patrones de estacionalidad.")
print("- Primer semestre (Enero-Junio): Post-navidad, primavera")
print("- Segundo semestre (Julio-Diciembre): Verano, otoño, temporada navideña")

print("\nEXPECTATIVAS PREVIAS:")
print("1. Mayor volumen de ventas en segundo semestre (temporada navideña)")
print("2. Productos diferentes tendrán demanda en cada semestre")
print("3. Ticket promedio mayor en S2 (compras de regalos)")

# Análisis comparativo
s1_sales = df_semester_1['TotalPrice'].sum()
s2_sales = df_semester_2['TotalPrice'].sum()
s1_avg = df_semester_1.groupby('InvoiceNo')['TotalPrice'].sum().mean()
s2_avg = df_semester_2.groupby('InvoiceNo')['TotalPrice'].sum().mean()

print("\nRESULTADOS:")
print(f"Semestre 1: {len(df_semester_1):,} transacciones | Ventas: £{s1_sales:,.2f} | Ticket: £{s1_avg:.2f}")
print(f"Semestre 2: {len(df_semester_2):,} transacciones | Ventas: £{s2_sales:,.2f} | Ticket: £{s2_avg:.2f}")

# ============================================
# PARTICIÓN 2: GEOGRÁFICA (UK vs Internacional)
# ============================================

print("\n" + "="*70)
print("PARTICIÓN 2: DIVISIÓN GEOGRÁFICA (UK vs INTERNACIONAL)")
print("="*70)

# Dividir en UK vs resto del mundo
df_uk = df[df['Country'] == 'United Kingdom'].copy()
df_international = df[df['Country'] != 'United Kingdom'].copy()

print("\nJUSTIFICACIÓN:")
print("División geográfica para distinguir mercado doméstico vs internacional.")
print("- UK: mercado principal local")
print("- Internacional: expansión y diversificación de mercados")

print("\nEXPECTATIVAS PREVIAS:")
print("1. UK tendrá mayor volumen de transacciones (mercado principal)")
print("2. Internacional puede tener tickets promedio más altos (compras al por mayor)")
print("3. Productos populares diferirán entre mercados")

# Análisis comparativo
uk_sales = df_uk['TotalPrice'].sum()
int_sales = df_international['TotalPrice'].sum()
uk_avg = df_uk.groupby('InvoiceNo')['TotalPrice'].sum().mean()
int_avg = df_international.groupby('InvoiceNo')['TotalPrice'].sum().mean()

print("\nRESULTADOS:")
print(f"UK: {len(df_uk):,} transacciones | Ventas: £{uk_sales:,.2f} | Ticket: £{uk_avg:.2f}")
print(f"Internacional: {len(df_international):,} transacciones | Ventas: £{int_sales:,.2f} | Ticket: £{int_avg:.2f}")
print(f"Países internacionales: {df_international['Country'].nunique()}")

# ============================================
# VISUALIZACIONES
# ============================================

# GRÁFICA 1: PARTICIÓN TEMPORAL
fig1, axes1 = plt.subplots(2, 2, figsize=(15, 10))
fig1.suptitle('PARTICIÓN 1: ANÁLISIS TEMPORAL', fontsize=16, fontweight='bold')

# Ventas por semestre
semester_sales = df.groupby('Year_Semester')['TotalPrice'].sum().sort_index()
axes1[0, 0].bar(semester_sales.index, semester_sales.values, color=['#3498db', '#e74c3c'])
axes1[0, 0].set_title('Ventas por Semestre')
axes1[0, 0].set_ylabel('Ventas (£)')
axes1[0, 0].tick_params(axis='x', rotation=45)
axes1[0, 0].grid(axis='y', alpha=0.3)

# Transacciones mensuales
monthly_trans = df.groupby(df['InvoiceDate'].dt.to_period('M')).size()
axes1[0, 1].plot(range(len(monthly_trans)), monthly_trans.values, marker='o', color='#2ecc71')
axes1[0, 1].set_title('Transacciones Mensuales')
axes1[0, 1].set_ylabel('Transacciones')
axes1[0, 1].grid(alpha=0.3)

# Comparación S1 vs S2
metrics = ['Transacciones', 'Ventas', 'Ticket Promedio']
s1_vals = [len(df_semester_1)/1000, s1_sales/1000, s1_avg]
s2_vals = [len(df_semester_2)/1000, s2_sales/1000, s2_avg]
x = np.arange(len(metrics))
width = 0.35
axes1[1, 0].bar(x - width/2, s1_vals, width, label='S1', color='#3498db')
axes1[1, 0].bar(x + width/2, s2_vals, width, label='S2', color='#e74c3c')
axes1[1, 0].set_title('Comparación S1 vs S2')
axes1[1, 0].set_xticks(x)
axes1[1, 0].set_xticklabels(metrics)
axes1[1, 0].legend()
axes1[1, 0].grid(axis='y', alpha=0.3)

# Ventas por mes
monthly_sales = df.groupby(df['InvoiceDate'].dt.month)['TotalPrice'].sum()
colors_month = ['#3498db']*6 + ['#e74c3c']*6
axes1[1, 1].bar(range(1, 13), monthly_sales.values, color=colors_month, alpha=0.7)
axes1[1, 1].set_title('Ventas por Mes')
axes1[1, 1].set_xlabel('Mes')
axes1[1, 1].set_ylabel('Ventas (£)')
axes1[1, 1].axvline(x=6.5, color='black', linestyle='--', linewidth=2)
axes1[1, 1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('particion_temporal.png', dpi=300, bbox_inches='tight')
plt.show()

# GRÁFICA 2: PARTICIÓN GEOGRÁFICA
fig2, axes2 = plt.subplots(2, 2, figsize=(15, 10))
fig2.suptitle('PARTICIÓN 2: ANÁLISIS GEOGRÁFICO', fontsize=16, fontweight='bold')

# Pie chart ventas
geo_data = [uk_sales, int_sales]
geo_labels = ['UK', 'Internacional']
axes2[0, 0].pie(geo_data, labels=geo_labels, autopct='%1.1f%%', colors=['#3498db', '#e74c3c'])
axes2[0, 0].set_title('Distribución de Ventas')

# Comparación métricas
metrics_geo = ['Transacciones', 'Ventas', 'Ticket Promedio']
uk_vals = [len(df_uk)/1000, uk_sales/1000, uk_avg]
int_vals = [len(df_international)/1000, int_sales/1000, int_avg]
x = np.arange(len(metrics_geo))
axes2[0, 1].bar(x - width/2, uk_vals, width, label='UK', color='#3498db')
axes2[0, 1].bar(x + width/2, int_vals, width, label='Internacional', color='#e74c3c')
axes2[0, 1].set_title('Comparación UK vs Internacional')
axes2[0, 1].set_xticks(x)
axes2[0, 1].set_xticklabels(metrics_geo)
axes2[0, 1].legend()
axes2[0, 1].grid(axis='y', alpha=0.3)

# Top países
top_countries = df_international['Country'].value_counts().head(10)
axes2[1, 0].barh(range(len(top_countries)), top_countries.values, color='#e74c3c')
axes2[1, 0].set_yticks(range(len(top_countries)))
axes2[1, 0].set_yticklabels(top_countries.index, fontsize=9)
axes2[1, 0].set_title('Top 10 Países Internacionales')
axes2[1, 0].set_xlabel('Transacciones')
axes2[1, 0].invert_yaxis()
axes2[1, 0].grid(axis='x', alpha=0.3)

# Evolución mensual UK vs Internacional
monthly_uk = df_uk.groupby(df_uk['InvoiceDate'].dt.to_period('M'))['TotalPrice'].sum()
monthly_int = df_international.groupby(df_international['InvoiceDate'].dt.to_period('M'))['TotalPrice'].sum()
axes2[1, 1].plot(range(len(monthly_uk)), monthly_uk.values, marker='o', label='UK', color='#3498db')
axes2[1, 1].plot(range(len(monthly_int)), monthly_int.values, marker='s', label='Internacional', color='#e74c3c')
axes2[1, 1].set_title('Ventas Mensuales')
axes2[1, 1].set_ylabel('Ventas (£)')
axes2[1, 1].legend()
axes2[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.savefig('particion_geografica.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✅ Análisis completado. Gráficas guardadas.")